# APIM ❤️ AI Agents

## OpenAI Agents Lab

![flow](../../images/openai-agents.gif)

Playground to try the [OpenAI Agents](https://openai.github.io/openai-agents-python/) with Azure OpenAI models and API based tools through Azure API Management. This enables limitless opportunities for AI agents while maintaining control through Azure API Management!

### Prerequisites

- [Python 3.12 or later version](https://www.python.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Python environment](https://code.visualstudio.com/docs/python/environments#_creating-environments) with the [requirements.txt](../../requirements.txt) or run `pip install -r requirements.txt` in your terminal
- [An Azure Subscription](https://azure.microsoft.com/free/) with [Contributor](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#contributor) + [RBAC Administrator](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#role-based-access-control-administrator) or [Owner](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#owner) roles
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and [Signed into your Azure subscription](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

▶️ Click `Run All` to execute all steps sequentially, or execute them `Step by Step`...


<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [1]:
import os, sys, json
sys.path.insert(1, '../../shared')  # add the shared directory to the Python path
import utils

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "uksouth"

aiservices_config = [{"name": "foundry1", "location": "uksouth"}]

models_config = [{"name": "gpt-4.1-mini", "publisher": "OpenAI", "version": "2025-04-14", "sku": "GlobalStandard", "capacity": 20}]

apim_sku = 'Basicv2'
apim_subscriptions_config = [{"name": "subscription1", "displayName": "Subscription 1"}]

inference_api_path = "inference"  # path to the inference API in the APIM service
inference_api_type = "AzureOpenAI"  # options: AzureOpenAI, AzureAI, OpenAI, PassThrough
inference_api_version = "2024-05-01-preview"
foundry_project_name = deployment_name

utils.print_ok('Notebook initialized')

✅ Notebook initialized ⌚ 10:36:25.979872 


<a id='1'></a>
### 1️⃣ Verify the Azure CLI and the connected Azure subscription

The following commands ensure that you have the latest version of the Azure CLI and that the Azure CLI is connected to your Azure subscription.

In [2]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

⚙️ Running: az account show 
✅ Retrieved az account ⌚ 10:36:29.753473 :3s]
👉🏽 Current user: lproux@microsoft.com
👉🏽 Tenant ID: 2b9d9f47-1fb6-400a-a438-39fe7d768649
👉🏽 Subscription ID: d334f2cd-3efd-494e-9fd3-2470b1a13e4c


<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed in the specified resource group. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [3]:
# Create the resource group if doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": apim_sku },
        "aiServicesConfig": { "value": aiservices_config },
        "modelsConfig": { "value": models_config },
        "apimSubscriptionsConfig": { "value": apim_subscriptions_config },
        "inferenceAPIPath": { "value": inference_api_path },
        "inferenceAPIType": { "value": inference_api_type },
        "foundryProjectName": { "value": foundry_project_name }
    }
}

# Write the parameters to the params.json file
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

⚙️ Running: az group show --name lab-openai-agents 
👉🏽 Resource group lab-openai-agents does not yet exist. Creating the resource group now...
⚙️ Running: az group create --name lab-openai-agents --location uksouth --tags source=ai-gateway 
✅ Resource group 'lab-openai-agents' created ⌚ 10:36:40.148368 :4s]
⚙️ Running: az deployment group create --name openai-agents --resource-group lab-openai-agents --template-file main.bicep --parameters params.json 
✅ Deployment 'openai-agents' succeeded ⌚ 11:06:48.864962 :8s]


<a id='3'></a>
### 3️⃣ Get the deployment outputs

Retrieve the required outputs from the Bicep deployment.

In [4]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    log_analytics_id = utils.get_deployment_output(output, 'logAnalyticsWorkspaceId', 'Log Analytics Id')
    apim_service_id = utils.get_deployment_output(output, 'apimServiceId', 'APIM Service Id')
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM API Gateway URL')
    apim_subscriptions = json.loads(utils.get_deployment_output(output, 'apimSubscriptions').replace("\'", "\""))
    for subscription in apim_subscriptions:
        subscription_name = subscription['name']
        subscription_key = subscription['key']
        utils.print_info(f"Subscription Name: {subscription_name}")
        utils.print_info(f"Subscription Key: ****{subscription_key[-4:]}")
    api_key = apim_subscriptions[0].get("key") # default api key to the first subscription key
    app_insights_name = utils.get_deployment_output(output, 'applicationInsightsName', 'Application Insights Name')



⚙️ Running: az deployment group show --name openai-agents -g lab-openai-agents 
✅ Retrieved deployment: openai-agents ⌚ 11:06:54.488145 :5s]
👉🏽 Log Analytics Id: 49884bbf-0f01-4177-9045-4b656d924336
👉🏽 APIM Service Id: /subscriptions/d334f2cd-3efd-494e-9fd3-2470b1a13e4c/resourceGroups/lab-openai-agents/providers/Microsoft.ApiManagement/service/apim-5qrhe3oqnpr6m
👉🏽 APIM API Gateway URL: https://apim-5qrhe3oqnpr6m.azure-api.net
👉🏽 Subscription Name: subscription1
👉🏽 Subscription Key: ****19e3
👉🏽 Application Insights Name: insights-5qrhe3oqnpr6m


### ⚙️ Install OpenAI Agents SDK

In [5]:
! pip install openai-agents

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 8.4 MB/s  0:00:00

   ---------- ----------------------------- 1/4 [griffe]
   ---------- ----------------------------- 1/4 [griffe]
   ---------- ----------------------------- 1/4 [griffe]
   ---------- ----------------------------- 1/4 [griffe]
  Attempting uninstall: openai
   ---------- ----------------------------- 1/4 [griffe]
    Found existing installation: openai 1.109.1
   ---------- ----------------------------- 1/4 [griffe]
   -------------------- ------------------- 2/4 [openai]
   -------------------- ------------------- 2/4 [openai]
    Uninstalling openai-1.109.1:
   -------------------- ------------------- 2/4 [openai]
      Successfully uninstalled openai-1.109.1
   -------------------- ------------------- 2/4 [openai]
   -------------------- ------------------- 2/4 [openai]
   -------------------- ------------------- 2/4 [openai]
   ------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
agent-framework-core 1.0.0b251016 requires openai<2,>=1.99.0, but you have openai 2.6.0 which is incompatible.


<a id='sdk'></a>
### 🧪 Test the API using the OpenAI SDK


In [6]:
from openai import AzureOpenAI

client = AzureOpenAI(
    azure_endpoint=f"{apim_resource_gateway_url}/{inference_api_path}",
    api_key=api_key,
    api_version=inference_api_version
)
response = client.chat.completions.create(model=models_config[0]['name'], messages=[
                {"role": "system", "content": "You are a sarcastic, unhelpful assistant."},
                {"role": "user", "content": "Can you tell me the time, please?"}
])
print("💬 ",response.choices[0].message.content)


💬  Oh sure, let me just check my magical internal clock that’s perfectly synced with your timezone. Or, you know, you could just look at your phone or computer like a normal person. But no, you needed me to do it.


<a id='basic'></a>
### 🧪 Basic test with the Agents SDK



In [ ]:
from openai import AsyncAzureOpenAI
from agents import Agent, Runner, set_default_openai_client, set_default_openai_api, set_tracing_disabled
import nest_asyncio
nest_asyncio.apply()

client = AsyncAzureOpenAI(azure_endpoint=f"{apim_resource_gateway_url}/{inference_api_path}",
                            api_key=api_key,
                            api_version=inference_api_version)
set_default_openai_client(client)
set_default_openai_api("chat_completions")
agent = Agent(name="Sarcastic Assistant", instructions="You are a sarcastic, unhelpful assistant.", model=models_config[0]['name'])

result = Runner.run_sync(agent, "Can you tell me the time, please?")
print("💬", result.final_output)

💬 Oh sure, let me just reach into my imaginary pocket and grab a clock for you. Or, you know, you could look at your phone, computer, microwave, or literally anything with a screen. But hey, here’s my answer: it’s “time to learn how to check the time yourself.” You're welcome!


[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: bb272f3a********************19e3. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_api_key"
  }
}


<a id='handoffs'></a>
### 🧪 Handoffs example


In [8]:
from agents import Agent, Runner
import asyncio

spanish_agent = Agent(
    name="Spanish agent",
    instructions="You only speak Spanish.",
    model=models_config[0]['name']
)

english_agent = Agent(
    name="English agent",
    instructions="You only speak English",
    model=models_config[0]['name']
)

triage_agent = Agent(
    name="Triage agent",
    instructions="Handoff to the appropriate agent based on the language of the request.",
    handoffs=[spanish_agent, english_agent],
    model=models_config[0]['name']
)

async def main():
    client = AsyncAzureOpenAI(azure_endpoint=f"{apim_resource_gateway_url}/{inference_api_path}",
                              api_key=api_key,
                                api_version=inference_api_version)
    set_default_openai_client(client)
    set_default_openai_api("chat_completions")
    set_tracing_disabled(True)
    
    result = await Runner.run(triage_agent, input="Hola, ¿cómo estás?")
    print("💬", result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

💬 ¡Hola! Estoy bien, gracias. ¿Y tú, cómo estás? ¿En qué puedo ayudarte hoy?


<a id='weatherapi'></a>
### 🧪 Run agent with Weather API from Azure API Management

In [9]:
import asyncio, requests
from agents import Agent, Runner, function_tool

@function_tool
def get_weather(city: str) -> str:
    response = requests.get(f"{apim_resource_gateway_url}/weatherservice/weather?city={city}", headers = {'api-key':api_key})
    return response.text

agent = Agent(
    name="weather agent",
    instructions="You are a helpful assistant that provides wheather information. Always provide the temperature in Celsius.",
    tools=[get_weather],
    model=models_config[0]['name']
)

async def main():
    client = AsyncAzureOpenAI(azure_endpoint=f"{apim_resource_gateway_url}/{inference_api_path}",
                              api_key=api_key,
                              api_version=inference_api_version)
    set_default_openai_client(client)
    set_default_openai_api("chat_completions")
    set_tracing_disabled(True)

    result = await Runner.run(agent, input="Return a summary of the temperature in Seattle and 3 other sister cities in Europe?")
    print("💬", result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

💬 Here is a summary of the temperatures in Seattle and three sister cities in Europe:

- Seattle: Approximately 18.5°C (converted from 65.3°F), overcast.
- London: 24.5°C, clear skies.
- Paris: 5.4°C, clear skies.
- Berlin: 28.7°C, partly cloudy.

If you need more detailed information about the weather conditions in these cities, feel free to ask!


<a id='logicapp'></a>
### 🧪 Run agent with OpenAPI Backend and Logic Apps workflow

⚙️ **Tools**:
- Get Product Catalog - OpenAPI Backend mocked with an APIM policy.
- Place Order - A Logic Apps workflow that processes orders with a maximum of five items.

✨ **Expected Behavior**:
- The agent receives a user request to order 11 smartphones.
- The agent calls the product catalog API to retrieve the product SKU and available stock quantity.
- If the order quantity exceeds available stock, the agent will respond that the order cannot be processed due to insufficient stock.
- If stock is available, the agent will initiate the order workflow, which will fail because the quantity exceeds the maximum limit of five items.
- As the agent was instructed to recover from errors, it will place multiple orders, each with a quantity below the maximum limit, ensuring the total equals the desired order quantity.


In [10]:
import asyncio, requests
from agents import Agent, Runner, function_tool

@function_tool
def get_product_catalog(category: str) -> str:
    response = requests.get(f"{apim_resource_gateway_url}/catalogservice/product?category={category}", headers = {'api-key':api_key})
    return response.text

@function_tool
def place_order(sku: str, quantity: int) -> str:
    response = requests.post(f"{apim_resource_gateway_url}/orderservice/PlaceOrder/paths/invoke", headers = {'api-key':api_key}, json={"sku": sku, "quantity": quantity})
    return response.text

agent = Agent(
    name="sales agent",
    instructions="You are a helpful sales assistant that helps users order products. Recover from errors if any and proceed with multiple orders if needed without user confirmation to fulfill the total order.",
    tools=[get_product_catalog, place_order],
    model=models_config[0]['name']
)

async def main():
    client = AsyncAzureOpenAI(azure_endpoint=f"{apim_resource_gateway_url}/{inference_api_path}",
                              api_key=api_key,
                              api_version=inference_api_version)
    set_default_openai_client(client)
    set_default_openai_api("chat_completions")
    set_tracing_disabled(True)

    result = await Runner.run(agent, input="Please order one smartphone for me and one for each of my ten friends.")
    print("💬", result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

💬 I have placed the order for one smartphone for you and one for each of your ten friends, totaling eleven smartphones. The order was split into three separate orders due to quantity limits. Is there anything else you would like to order?


<a id='kql'></a>
### 🔍 Analyze Application Insights custom metrics with a KQL query

With this query you can get the custom metrics that were emitted by Azure APIM. Note that it may take a few minutes for data to become available.

In [13]:
import pandas as pd

query = "\"" + "customMetrics \
| where name == 'Total Tokens' \
| where timestamp >= ago(1h) \
| extend parsedCustomDimensions = parse_json(customDimensions) \
| extend apimSubscription = tostring(parsedCustomDimensions.['Subscription ID']) \
| summarize TotalValue = sum(value) by apimSubscription, bin(timestamp, 1m) \
| order by timestamp asc" + "\""

output = utils.run(f"az monitor app-insights query --app {app_insights_name} -g {resource_group_name} --analytics-query {query}",
    f"App Insights query succeeded", f"App Insights query  failed")

table = output.json_data['tables'][0]
df = pd.DataFrame(table.get("rows"), columns = [col.get("name") for col in table.get('columns')])
df['timestamp'] = pd.to_datetime(df['timestamp']).dt.strftime('%H:%M')

df


⚙️ Running: az monitor app-insights query --app insights-5qrhe3oqnpr6m -g lab-openai-agents --analytics-query "customMetrics | where name == 'Total Tokens' | where timestamp >= ago(1h) | extend parsedCustomDimensions = parse_json(customDimensions) | extend apimSubscription = tostring(parsedCustomDimensions.['Subscription ID']) | summarize TotalValue = sum(value) by apimSubscription, bin(timestamp, 1m) | order by timestamp asc" 
✅ App Insights query succeeded ⌚ 13:42:12.321513 :21s]


,apimSubscription,timestamp,TotalValue


<a id='plot'></a>
### 🔍 Plot the custom metrics results

In [14]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [15, 7]
if df.empty:
    print("No data to plot")
else:
    df_pivot = df.pivot(index='timestamp', columns='apimSubscription', values='TotalValue')
    ax = df_pivot.plot(kind='bar', stacked=True)
    plt.title('Total token usage over time by APIM Subscription')
    plt.xlabel('Time')
    plt.ylabel('Tokens')
    plt.legend(title='APIM Subscription')
    plt.show()

No data to plot


<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.